In [ ]:
class Cell:
    def __init__(self, combustion_temp, heat_resistance, dryness):
        self.combustion_temp = combustion_temp
        self.heat_resistance = heat_resistance
        self.dryness = dryness